<a href="https://colab.research.google.com/github/BaronVonBussin/NewTransit/blob/main/turbulance_other.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Load sector data - you'll need these 11 sectors:
sectors = [
    'XLB',  # Materials
    'XLE',  # Energy
    'XLF',  # Financials
    'XLI',  # Industrials
    'XLK',  # Technology
    'XLP',  # Consumer Staples
    'XLU',  # Utilities
    'XLV',  # Healthcare
    'XLY',  # Consumer Discretionary
]

# Load data for each sector (adjust path)
def load_sector_data(symbols, start='2010-07-01', end='2019-12-31'):
    df = pd.DataFrame()
    for symbol in symbols:
        data = pd.read_csv(f'{symbol}_OHLC.csv', thousands=',')
        data.index = pd.to_datetime(data['Date'])
        df[symbol] = data['Close'].pct_change()
    return df.dropna()

returns = load_sector_data(sectors)

def calculate_turbulence(returns, window=252):
    turbulence = []
    for i in range(window, len(returns)):
        historical = returns.iloc[i-window:i]
        mu = historical.mean()
        sigma = historical.cov()
        sigma_inv = np.linalg.inv(sigma)
        y = returns.iloc[i] - mu
        d = np.sqrt(y.dot(sigma_inv).dot(y))
        turbulence.append(d)

    return pd.Series(turbulence, index=returns.index[window:])

turb_index = calculate_turbulence(returns)
turb_index.to_csv('sector_turbulence_index.csv')

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load data starting 6 months before required start date
start_date = '2010-07-01'  # 6 months before 2011
end_date = '2019-12-31'

# Load your data (adjust paths)
sp = pd.read_csv('path_to_spx.csv', thousands=',')
vx = pd.read_csv('path_to_vix.csv', thousands=',')

# Prepare returns dataframe
sp.index = pd.to_datetime(sp['date'])
vx.index = pd.to_datetime(vx['date'])

# Calculate daily returns
returns = pd.DataFrame({
   'SPX': sp['close'].pct_change(),
   'VIX': vx['close'].pct_change()
}).dropna()

def calculate_turbulence(returns, window=252):
   turbulence = []
   for i in range(window, len(returns)):
       # Get historical sample
       historical = returns.iloc[i-window:i]

       # Calculate mean and covariance
       mu = historical.mean()
       sigma = historical.cov()
       sigma_inv = np.linalg.inv(sigma)

       # Get current return vector
       y = returns.iloc[i] - mu

       # Calculate Mahalanobis distance
       d = np.sqrt(y.dot(sigma_inv).dot(y))
       turbulence.append(d)

   return pd.Series(turbulence, index=returns.index[window:])

# Calculate turbulence index
turbulence_index = calculate_turbulence(returns)

# Save to CSV
turbulence_index.to_csv('turbulence_index_2011_2019.csv', header=['turbulence'])

print(f"Turbulence index generated from {turbulence_index.index[0]} to {turbulence_index.index[-1]}")